## Openaire API

In [1]:
openaire_api_endpoint = "http://api.openaire.eu/search/publications"
parameters = {"size" : "3", "format" : "json", 
               "author": "Fuwen Wei, Yonggang Nie, Lei Shan; YiBo Hu"} #"title" : "xxx",

In [ ]:
import requests
response = requests.get(openaire_api_endpoint,
                        params=parameters)

In [ ]:
import json
response_dict = json.loads(response.text)
#print(response_dict['response']['results']['result'][0]['metadata'])

In [ ]:
oaf_result_entity = response_dict['response']['results']['result'][0]['metadata']['oaf:entity']['oaf:result']

In [ ]:
oaf_result_entity.keys()
#oaf_result_entity[ 'publisher']

In [ ]:
oaf_result_entity['pid']
#oaf_result_entity['pid']['$']

In [ ]:
oaf_result_entity['originalId'] #Contains DOI and ISSN!

In [ ]:
oaf_result_entity['title'] # Maybe have some sensible string comparison algorithm

In [ ]:
oaf_result_entity['creator']

In [ ]:
oaf_result_entity['description']

## API OAI

In [ ]:
from sickle import Sickle

In [ ]:
sickle = Sickle(endpoint="https://data.mendeley.com/oai", http_method="GET", max_retries= 2, 
                protocol_version="2.0", default_retry_after=3)
records = sickle.ListRecords(metadataPrefix='oai_dc')

In [ ]:
id = sickle.Identify()

In [ ]:
id.granularity

## CROSSREF

Note: Postponed; xml query only allows lookup of doi from metadata; might be useful (ABSTRACT is just maybe included ) https://github.com/Crossref/rest-api-doc/blob/master/api_format.md

In [ ]:
query = "https://doi.crossref.org/openurl/?pid=mw784@cam.ac.uk&id=doi:10.1103/PhysRev.47.777&noredirect=true"
response = requests.get(query)

In [ ]:
from habanero import Crossref
cr = Crossref()
#cr.journals(ids = "0071-3260", works = True)

In [ ]:
res = cr.journals(ids = "2167-8359", works = True, cursor = "*", cursor_max = 200, progress_bar = True)

In [ ]:
cr.types(works = True)

In [ ]:
journ = cr.journals(works=False, query="Evolutionary Biology", limit=50, progress_bar=True)['message']['items']  #['items-per-page'] #50 i

In [ ]:
journ[0]['title']
journ[0]['publisher']
journ[0]['issn-type']
journ[0]['ISSN']

In [ ]:
print("\n".join([j['title'] + " published by " + j['publisher'] 
                 + " has ID: " + " ".join(j['ISSN']) for j in journ]))

In [ ]:
j_works = cr.journals(ids=["1934-2845"], limit=100, select=["title", "DOI", "issue", "volume"], sort='published-print', order='desc', filter = {'from_pub_date': '2019-01-01'}, works=True, progress_bar=True)

In [ ]:
print(f"There are {j_works['message']['total-results']} results... \n")
print("Displaying 20 Titles: \n")
works_data =j_works['message']['items']
works_data[20]['volume']
print(" ".join([f""""{w.get('title')[0]}";; Vol: {w.get('volume', None)} Iss: {w.get('issue', None)} \n""" for w in works_data]))

Get Throughput for today

In [ ]:
cr = requests.head("https://api.crossref.org/")
int(cr.headers['x-rate-limit-limit']) / int(cr.headers['x-rate-limit-interval'][:-1])

Perform a keyword query for an article

In [1]:
from habanero import Crossref
cr = Crossref(mailto="markus.walder00@gmail.com")
to_select = ["abstract", "title", "original-title", "issue", "short-title", "DOI", "published-print", "volume", "author", 'URL', 'ISSN']

In [3]:
query_string = "aerhdghsf"
res = cr.works(limit=5, query_bibliographic=query_string,# query_author="Gluck Buglion",
        filter={"has-abstract":True, "type":"journal-article", "from-print-pub-date":"2018-01" , "until-print-pub-date" :"2019-01" },
        select= to_select)

In [4]:
res.keys() 

dict_keys(['status', 'message-type', 'message-version', 'message'])

In [3]:
doi = "10.1386/eme.17.4.465_1"
doi_response =cr.works(ids = doi)

In [17]:
doi_response['message'].get('issued')['date-parts'][0]

[2018, 12, 1]

In [7]:
import habanero
import requests
nonsense_doi = "10.1386/eme.17.334.465_1"
try:
    nonsense_doi_response =cr.works(ids = nonsense_doi)
except requests.exceptions.HTTPError as e:
    print("Error")

Error


In [21]:
nonsense_query = "nonsense"
resp =cr.works(limit=1, query_bibliographic=nonsense_query)

In [22]:
resp["message"]

{'facets': {},
 'total-results': 6036,
 'items': [{'indexed': {'date-parts': [[2020, 7, 3]],
    'date-time': '2020-07-03T10:10:53Z',
    'timestamp': 1593771053300},
   'reference-count': 0,
   'publisher': 'Stanford University Press',
   'isbn-type': [{'value': '9780804783064', 'type': 'electronic'}],
   'content-domain': {'domain': [], 'crossmark-restriction': False},
   'published-print': {'date-parts': [[2020, 12, 31]]},
   'DOI': '10.1515/9780804783064-001',
   'type': 'book-chapter',
   'created': {'date-parts': [[2020, 7, 3]],
    'date-time': '2020-07-03T09:51:02Z',
    'timestamp': 1593769862000},
   'page': '1-14',
   'source': 'Crossref',
   'is-referenced-by-count': 0,
   'title': ['1. Introduction: Nonsense, Victorian Nonsense, Romantic Nonsense'],
   'prefix': '10.1515',
   'member': '374',
   'container-title': ['In Praise of Nonsense'],
   'link': [{'URL': 'https://www.degruyter.com/view/book/9780804783064/10.1515/9780804783064-001.xml',
     'content-type': 'text/html

In [13]:
import re
jats_regex = re.compile(r"<jats:\w*>|</jats:\w*>")#"|</jats(.)*>")
#jats_regex.sub("", doi_response['message'].get('abstract'))
jats_regex.sub("", doi_response['message'].get('abstract'))

'Photojournalism is a vital resource for thinking, feeling, expressing, acting and dealing with problems of collective living, argues Robert Hariman and John Louis Lucaites. It is a kind of ‘equipment for living’. This article proposes the concept rhetorical working through as a way of describing how photojournalistic rhetoric functions as ‘equipment for living’. It argues that humans constantly perform three kinds of rhetorical working through: dealing with issues and arguments, dealing with social relations and dealing with identity and self.'

Further Filters

from-pub-date	{date}	metadata where published date is since (inclusive) {date}
until-pub-date	{date}	metadata where published date is before (inclusive) {date}
from-online-pub-date	{date}	metadata where online published date is since (inclusive) {date}
until-online-pub-date	{date}	metadata where online published date is before (inclusive) {date}
from-print-pub-date	{date}	metadata where print published date is since (inclusive) {date}
until-print-pub-date	{date}	metadata where print published date is before (inclusive) {date}